# Enrollments View

In [0]:
CREATE OR REPLACE VIEW dev.bi.content_s12n_enollments_monthly_history_mgtv as
select 
to_date(date_trunc('month',phoenix_specialization_enrollment_ts)) as dt_month,
base_phoenix_specialization_id as content_id,
count(distinct user_id) as enrollments,
count(distinct CASE WHEN is_phoenix_specialization_enrollment_paid
        OR is_phoenix_specialization_enrollment_paid_cplus
        OR is_phoenix_specialization_enrollment_paid_clite THEN user_id end) as paid_enrollments
 FROM prod.gold.phoenix_specialization_enrollments_vw a
inner join dev.bi.content_dim_kpi b on a.base_phoenix_specialization_id = b.underlying_product_item_id
WHERE date(phoenix_specialization_enrollment_ts) < to_date(date_trunc('month',current_date))
and date(phoenix_specialization_enrollment_ts) >=to_date(date_trunc('month', add_months(current_date, -2)))
group by 1,2
;

In [0]:
DELETE FROM dev.bi.content_s12n_enollments_monthly_history_mgtf
WHERE dt_month >= to_date(date_trunc('month', add_months(current_date, -2)))

num_affected_rows
3332


In [0]:
INSERT INTO dev.bi.content_s12n_enollments_monthly_history_mgtf
select * from dev.bi.content_s12n_enollments_monthly_history_mgtv

num_affected_rows,num_inserted_rows
3332,3332


# Enrollments Final View

In [0]:
create or replace view dev.bi.content_s12n_enollments_monthly_history_view_mgtv as
with enrol_agg (select dt_month,
                  content_sub_domain, 
                  cluster_rank,
                  SUM(enrollments) AS enrollments,
                  SUM(paid_enrollments) AS paid_enrollments
                  from dev.bi.content_s12n_enollments_monthly_history_mgtf a
                  left join dev.bi.content_dim_kpi b on a.content_id = b.underlying_product_item_id
                  left join dev.bi.s12n_partner_cluster_ranks_mgtv c on b.partner_id = c.partner_id
                  where content_sub_domain is not null
                  group by 1,2,3)
,enrol_view (select dt_month,
                  content_sub_domain,  
                  cluster_rank,
                  row_number() OVER (PARTITION BY content_sub_domain,cluster_rank ORDER BY dt_month) AS row_num, 
                  SUM(enrollments) OVER (PARTITION BY content_sub_domain,cluster_rank ORDER BY dt_month ROWS BETWEEN 12 PRECEDING AND 1 PRECEDING) AS rolling_enrollments,
                  SUM(paid_enrollments) OVER (PARTITION BY content_sub_domain,cluster_rank ORDER BY dt_month ROWS BETWEEN 12 PRECEDING AND 1 PRECEDING) AS rolling_paid_enrollments
                  from enrol_agg)
select date(dt_month) as dt_month,
lower(replace(content_sub_domain,' ','-')) as content_sub_domain,
cluster_rank,
sum(rolling_enrollments) as sd_avg_rolling_enrollments,
sum(rolling_paid_enrollments) as sd_avg_rolling_paid_enrollments
from enrol_view
group by 1,2,3;